# Simple Nucleus Segmentation for Cervical Cancer Images

This notebook provides a streamlined approach to nucleus segmentation using Cellpose.

## Overview
- **Input**: Cervical cancer cell images (.bmp format)
- **Output**: Nucleus masks (.png format) in "Nucleus Masks" folders
- **Method**: Cellpose segmentation with post-processing

In [ ]:
# Import essential libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import time
from datetime import datetime

# Cellpose imports
from cellpose import models, core
from skimage import measure

print("✅ Libraries imported successfully")
print(f"🔥 GPU available: {core.use_gpu()}")

In [ ]:
# Configuration - Edit these paths as needed
DATASET_ROOT = r"c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Augmented Dataset - Limited Enhancement"
INPUT_FOLDER = "NLM_CLAHE"
OUTPUT_FOLDER = "Nucleus Masks"

CELL_CATEGORIES = [
    "im_Dyskeratotic",
    "im_Koilocytotic", 
    "im_Metaplastic",
    "im_Parabasal",
    "im_Superficial-Intermediate"
]

# Cellpose parameters
MODEL_TYPE = "nuclei"
DIAMETER = None  # Auto-detect
FLOW_THRESHOLD = 0.4
CELLPROB_THRESHOLD = 0.0
MIN_SIZE = 50   # Minimum nucleus size in pixels
MAX_SIZE = 5000 # Maximum nucleus size in pixels

print(f"📂 Dataset root: {DATASET_ROOT}")
print(f"🏷️ Categories: {len(CELL_CATEGORIES)}")
print(f"🔬 Model: {MODEL_TYPE}")

In [ ]:
# Initialize Cellpose model
print("🤖 Initializing Cellpose model...")
model = models.CellposeModel(gpu=core.use_gpu(), model_type=MODEL_TYPE)
print("✅ Model ready!")

In [ ]:
def segment_nucleus(image_path, output_path):
    """Segment nuclei in a single image and save the mask."""
    try:
        # Load image
        img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
        if img is None:
            return False, "Failed to load image"
        
        # Run Cellpose
        masks, flows, styles, diams = model.eval(
            img,
            diameter=DIAMETER,
            channels=[0, 0],  # Grayscale
            flow_threshold=FLOW_THRESHOLD,
            cellprob_threshold=CELLPROB_THRESHOLD
        )
        
        # Filter by size
        if masks.max() > 0:
            props = measure.regionprops(masks)
            filtered_mask = np.zeros_like(masks)
            
            for prop in props:
                if MIN_SIZE <= prop.area <= MAX_SIZE:
                    filtered_mask[masks == prop.label] = prop.label
            
            # Relabel consecutively
            masks = measure.label(filtered_mask > 0)
        
        # Save mask
        cv2.imwrite(str(output_path), masks.astype(np.uint16))
        
        # Return success and nucleus count
        num_nuclei = len(np.unique(masks)) - 1
        return True, num_nuclei
        
    except Exception as e:
        return False, str(e)

print("✅ Segmentation function defined")

In [ ]:
def process_category(category, limit=None, test_mode=False):
    """Process all images in a category."""
    print(f"\n📁 Processing {category}...")
    
    # Set up paths
    input_dir = Path(DATASET_ROOT) / category / INPUT_FOLDER
    output_dir = Path(DATASET_ROOT) / category / OUTPUT_FOLDER
    output_dir.mkdir(exist_ok=True)
    
    # Get image files
    image_files = sorted(list(input_dir.glob("*.bmp")))
    if not image_files:
        print(f"⚠️ No images found in {input_dir}")
        return []
    
    if limit:
        image_files = image_files[:limit]
        print(f"📊 Processing first {len(image_files)} images (test mode)")
    
    results = []
    
    # Process images with progress bar
    for img_path in tqdm(image_files, desc=category.replace("im_", "")):
        start_time = time.time()
        output_path = output_dir / f"{img_path.stem}_mask.png"
        
        success, result = segment_nucleus(img_path, output_path)
        processing_time = time.time() - start_time
        
        results.append({
            'category': category,
            'image': img_path.name,
            'success': success,
            'nuclei_count': result if success else 0,
            'error': result if not success else None,
            'time': processing_time
        })
        
        # Show progress for test mode
        if test_mode and success:
            print(f"  ✅ {img_path.name}: {result} nuclei ({processing_time:.1f}s)")
    
    # Summary
    successful = sum(1 for r in results if r['success'])
    total_nuclei = sum(r['nuclei_count'] for r in results if r['success'])
    avg_time = np.mean([r['time'] for r in results if r['success']]) if successful > 0 else 0
    
    print(f"✅ {category}: {successful}/{len(results)} images processed")
    print(f"🔬 Total nuclei found: {int(total_nuclei)}")
    print(f"⏱️ Average time: {avg_time:.1f}s per image")
    
    return results

print("✅ Processing function defined")

## Test on Sample Images

Let's first test the segmentation on a few images from each category:

In [ ]:
# Test with 2 images per category
print("🧪 Running test on sample images...")
test_results = []

for category in CELL_CATEGORIES:
    category_results = process_category(category, limit=2, test_mode=True)
    test_results.extend(category_results)

# Test summary
successful_tests = sum(1 for r in test_results if r['success'])
print(f"\n📊 Test Summary: {successful_tests}/{len(test_results)} images processed successfully")

if successful_tests > 0:
    print("✅ Test successful! Ready for full processing.")
else:
    print("❌ Test failed. Please check the configuration.")

## Visualize Test Results

Let's look at one test result to verify the segmentation quality:

In [ ]:
# Visualize one test result
if successful_tests > 0:
    # Find first successful result
    test_result = next(r for r in test_results if r['success'])
    category = test_result['category']
    image_name = test_result['image']
    
    # Load original image and mask
    img_path = Path(DATASET_ROOT) / category / INPUT_FOLDER / image_name
    mask_path = Path(DATASET_ROOT) / category / OUTPUT_FOLDER / f"{Path(image_name).stem}_mask.png"
    
    original = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    
    # Create visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    axes[0].imshow(original, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    # Nucleus mask
    axes[1].imshow(mask, cmap='nipy_spectral')
    axes[1].set_title(f'Nucleus Mask ({test_result["nuclei_count"]} nuclei)')
    axes[1].axis('off')
    
    # Overlay
    overlay = original.copy()
    overlay[mask > 0] = 255  # Highlight nuclei
    axes[2].imshow(overlay, cmap='gray')
    axes[2].set_title('Nuclei Highlighted')
    axes[2].axis('off')
    
    plt.suptitle(f'Test Result: {category} - {image_name}')
    plt.tight_layout()
    plt.show()
    
    print(f"📸 Showing results for: {image_name}")
    print(f"🔬 Nuclei detected: {test_result['nuclei_count']}")
    print(f"⏱️ Processing time: {test_result['time']:.1f}s")

## Full Dataset Processing

**⚠️ WARNING:** This will process all images in your dataset (thousands of images). 
Uncomment and run the cell below only when ready for full processing:

In [ ]:
# UNCOMMENT TO RUN FULL PROCESSING
# print("🚀 Starting full dataset processing...")
# start_time = time.time()
# all_results = []

# for category in CELL_CATEGORIES:
#     category_results = process_category(category)
#     all_results.extend(category_results)

# # Save results to CSV
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# results_df = pd.DataFrame(all_results)
# results_path = f"nucleus_segmentation_results_{timestamp}.csv"
# results_df.to_csv(results_path, index=False)

# # Final summary
# total_time = time.time() - start_time
# successful = sum(1 for r in all_results if r['success'])
# total_nuclei = sum(r['nuclei_count'] for r in all_results if r['success'])

# print(f"\n🎉 PROCESSING COMPLETE!")
# print(f"📊 Total images: {len(all_results)}")
# print(f"✅ Successful: {successful}")
# print(f"🔬 Total nuclei: {int(total_nuclei)}")
# print(f"⏱️ Total time: {total_time/3600:.1f} hours")
# print(f"💾 Results saved: {results_path}")

print("⚠️ Full processing is commented out for safety.")
print("📝 Uncomment the code above to process all images.")
print("💡 Expected time: Several hours depending on your system.")

## Results

After processing, you will have:

1. **Nucleus Masks**: PNG files in each category's "Nucleus Masks" folder
2. **CSV Results**: Detailed processing results with metrics
3. **Directory Structure**:
   ```
   Each Category/
   ├── NLM_CLAHE/           # Original images
   └── Nucleus Masks/       # Generated masks
       ├── image1_mask.png
       ├── image2_mask.png
       └── ...
   ```

### Key Features:
- **Simple & Fast**: Streamlined code without unnecessary complexity
- **Robust**: Size filtering removes artifacts
- **Progress Tracking**: Real-time progress bars
- **Error Handling**: Continues processing even if some images fail
- **Test Mode**: Verify on samples before full processing